# CARGA DEL DATASET Y LIBRERIAS

## Importacion de Librerias

In [13]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

## Carga de los datasets para su tratamiento

In [3]:
df_user_reviews_recommendacion_usuario_user_id = pd.read_parquet('../Data/df_user_reviews_recommendacion_usuario_user_id.parquet')
df_user_items_recommendacion_usuario_user_id = pd.read_parquet('../Data/df_user_items_recommendacion_usuario_user_id.parquet')

# EXPLORACION BASICA

## Explorando df_user_reviews_recommendacion_usuario_user_id

In [4]:
print(df_user_reviews_recommendacion_usuario_user_id.shape)
print('Tipos de las columnas:')
print(df_user_reviews_recommendacion_usuario_user_id.dtypes)
print('Datos faltantes:')
print(df_user_reviews_recommendacion_usuario_user_id.isnull().sum())
df_user_reviews_recommendacion_usuario_user_id.head(1)

(48498, 4)
Tipos de las columnas:
user_id               object
item_id               object
recommend               bool
sentiment_analysis     int64
dtype: object
Datos faltantes:
user_id               0
item_id               0
recommend             0
sentiment_analysis    0
dtype: int64


,user_id,item_id,recommend,sentiment_analysis
0,76561197970982479,1250,True,2


## Explorando df_user_items_recommendacion_usuario_user_id

In [5]:
print(df_user_items_recommendacion_usuario_user_id.shape)
print('Tipos de las columnas:')
print(df_user_items_recommendacion_usuario_user_id.dtypes)
print('Datos faltantes:')
print(df_user_items_recommendacion_usuario_user_id.isnull().sum())
df_user_items_recommendacion_usuario_user_id.head(1)

(5094082, 3)
Tipos de las columnas:
user_id      object
item_id      object
item_name    object
dtype: object
Datos faltantes:
user_id      0
item_id      0
item_name    0
dtype: int64


,user_id,item_id,item_name
0,76561197970982479,10,Counter-Strike


# TRANSFORMACION DE LOS ENDPOINT DATASETS DEL SISTEMA DE RECOMENDACION

## Transformaciones

### Merging de los datasetes

Agregamos el campo 'item_name' del dataset 'df_user_items_recommendacion_usuario_user_id' al dataset 'df_user_reviews_recommendacion_usuario_user_id'.

In [6]:
df_recommendacion_usuario_user_id = df_user_reviews_recommendacion_usuario_user_id.merge(df_user_items_recommendacion_usuario_user_id, on=['user_id', 'item_id'], how='left')

print(df_recommendacion_usuario_user_id.shape)
print('columnas en df_recommendacion_usuario_user_id: ','[' + ', '.join(df_recommendacion_usuario_user_id.keys()) + ']')
print(df_recommendacion_usuario_user_id.isnull().sum())
df_recommendacion_usuario_user_id.head(3)

(48498, 5)
columnas en df_recommendacion_usuario_user_id:  [user_id, item_id, recommend, sentiment_analysis, item_name]
user_id                   0
item_id                   0
recommend                 0
sentiment_analysis        0
item_name             12205
dtype: int64


,user_id,item_id,recommend,sentiment_analysis,item_name
0,76561197970982479,1250,True,2,Killing Floor
1,76561197970982479,22200,True,2,Zeno Clash
2,76561197970982479,43110,True,2,Metro 2033


### Tratamiento del dataset mergeado

El campo 'item_name' tiene 12205 valores faltantes. Se eliminan

In [7]:
df_recommendacion_usuario_user_id = df_recommendacion_usuario_user_id.dropna()
df_recommendacion_usuario_user_id.reset_index(drop=True, inplace=True)

print(df_recommendacion_usuario_user_id.shape)
print(df_recommendacion_usuario_user_id.isnull().sum())

(36293, 5)
user_id               0
item_id               0
recommend             0
sentiment_analysis    0
item_name             0
dtype: int64


Filtrar los registros donde 'recommend' es True y 'sentiment_analysis' es igual a 2

In [8]:
df_recommendacion_usuario_user_id = df_recommendacion_usuario_user_id[(df_recommendacion_usuario_user_id['recommend'] == True) & (df_recommendacion_usuario_user_id['sentiment_analysis'] == 2)]
print(df_recommendacion_usuario_user_id.shape)

(21959, 5)


Eliminamos los campos 'recommend' y 'sentiment_analysis'


In [9]:
df_recommendacion_usuario_user_id.drop(columns=['recommend', 'sentiment_analysis'], inplace=True)
print('columnas en df_recommendacion_usuario_user_id: ','[' + ', '.join(df_recommendacion_usuario_user_id.keys()) + ']')


columnas en df_recommendacion_usuario_user_id:  [user_id, item_id, item_name]


Checkamos 25 registros al azar

In [11]:
muestra = df_recommendacion_usuario_user_id.sample(n=100, random_state= 60).head(25) 
muestra

,user_id,item_id,item_name
15044,Croosoe,730,Counter-Strike: Global Offensive
1734,tickets845,224480,Octodad: Dadliest Catch
11369,WHOTHEFUCKTOOKMYURL,204300,Awesomenauts
34339,PuggyYo,304930,Unturned
29186,90s_cars,227300,Euro Truck Simulator 2
15128,stopreadingthisthing,304930,Unturned
16057,BillyDatKid,311190,Super Panda Adventures
30597,76561198060545297,218620,PAYDAY 2
17650,Ep1cbunny,214560,Mark of the Ninja
1543,Haelnorr,233450,Prison Architect


11369 is really pissed off!

## Guardamos los datasets preparados

In [21]:
df_recommendacion_usuario_user_id.to_parquet('../Data/df_recommendacion_usuario_user_id.parquet', index=False)